## Valuation of Kiwibank using multiples ##

The code below shows the valuation of Kiwibank at the moment of the announcement of the acquisition by the New Zealand government on 22 August 2022. I use a valuation method based on so-called "multiples", because traditional valuation approaches, based on discounting cashflows, do not work well. The discount factor is low and volatile, which will result in large valuation swings.

The approach I take is well-documented and empirically tested. See, for example, this academic paper: [Equity Valuation Using Multiples Jing Liu, Doron Nissim, Jacob Thomas](https://onlinelibrary.wiley.com/doi/10.1111/1475-679X.00042).

The approach relies on a group of comparable firms. These firms are very similar to the firm one wants to value. The difference is that the comparable firms are all listed, so they all have a market value. For these firms, it is straightforward to find a representative P/E ratio (Price to Earnings) or Price to Book ratio. To determine the value of an unlisted firm, you multiply these ratios by the earnings or book value of the firm you are interested in.

To illustrate: suppose the average P/E ratio of the group of comparable firms is 15 and your firm reports a profit of $\$1 million. Its value would then be $\$15 million.

The challenging part of this approach is finding the right group of comparable firms. For Kiwibank, I chose banks from OECD countries and European banks. I excluded banks from the US and Japan (because the accounting is different in the US, and because there are too many Japanese banks ending up in my sample). I also selected banks that are about the same size as Kiwibank and I filtered out banks with extreme values of critical variables.

After determining the proper group of comparable banks, I determined the P/E ratio and the P/B (Price to Book) ratio and multiplied these ratios with Kiwibank's income number and book value.

<br>

I use Python to show you my workings. The data is from Datastream and the variables I use are shown below. You should be able to replicate my work.

Initiate Python, variables, countries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
variables = {'X(MV)~E': 'MV', 'PE': 'PE', 'X(WC05476)~E': 'BVPS', 'MTBV': 'MTBV', 'PTBV': 'PTBV', 'X(DWTA)~E': 'DWTA', 'X(EPS)~E': 'EPS', 'X(P)~E': 'P', 'NOSH': 'NOSH', 'X(WC01751)~E': 'NI', 'NAME': 'NAME', 'country_name': 'country', 'X(WC18228)~E': 'Tier1', 'WC18157': 'Tier1Ratio', 'X(WC18156)~E': 'RWA', 'date': 'date'} # All values retrieved in Euros
country_list = pd.read_csv('oecd_eu_oz_nz.csv').set_index('country')

Retrieve data from a Datastream Request table

In [2]:
df = pd.read_excel("DFORequestTable2.xlsm", sheet_name="values").set_index('GGISO').join(country_list).rename(columns=variables).set_index('Type').dropna(subset=['country', 'EPS', 'DWTA', 'MTBV', 'NI', 'PE'])
df = df.assign(dual=df.NAME.str.contains('\([A-Z]+\)', regex=True, na=False),
               NOSH=df.NOSH.replace(0, np.NaN))

keep = ['NAME', 'country', 'NOSH', 'PE', 'MTBV', 'PTBV', 'NI', 'RWA', 'Tier1Ratio', 'MV', 'DWTA', 'EPS', 'P', 'BVPS', 'Tier1', 'date']

Determine additional variables and make some adjustments.

In [3]:
def bv_calc(df):
    return df.NOSH.mul(df.BVPS)


df = df[keep].assign(BV=bv_calc(df),
               MV=df.MV.mul(1000),  # mv in millions  bv in thousands
               leverage=bv_calc(df).div(df['DWTA']),
               roe=df['EPS'].div(df['BVPS']).replace(0, np.NaN),
               roa=df['NI'].div(df['DWTA']).replace(0, np.NaN),
               density=df['RWA'].div(df['DWTA']),
               Tier1Ratio=df['Tier1Ratio'].div(100)
               ).drop('TK:TBB').loc[df.dual == False]

df.sample(5)

,NAME,country,NOSH,PE,MTBV,PTBV,NI,RWA,Tier1Ratio,MV,...,EPS,P,BVPS,Tier1,date,BV,leverage,roe,roa,density
Type,,,,,,,,,,,,,,,,,,,,,
F:CAIV,CARDE.CAMU.IEV.,France,2277.0,8.3,0.14,0.20,61508.0,5457579.0,0.2068,157110.0,...,8.350,68.9900,349.480,1128000.0,2021-12-31,7.957660e+05,0.044912,0.023893,0.003471,0.308016
PO:ALR,ALIOR BANK,Poland,130554.0,14.2,1.21,1.21,105147.0,10782315.0,0.1255,1558100.0,...,0.840,11.9300,9.892,1352726.0,2021-12-31,1.291440e+06,0.071273,0.084917,0.005803,0.595062
F:LAV,CRCAM ATLANTIQUE VENDEE,France,1260.0,6.8,0.12,0.18,113463.0,NaN,0.2648,111670.0,...,13.020,88.6000,482.581,NaN,2021-12-31,6.080521e+05,0.021718,0.026980,0.004053,NaN
E:SAN,BANCO SANTANDER,Spain,17340620.0,8.3,0.59,0.58,8124000.0,579478000.0,0.1379,50990120.0,...,0.350,2.9405,5.095,79939000.0,2021-12-31,8.835046e+07,0.056046,0.068695,0.005154,0.367597
TK:FIN,QNB FINANSBANK B,Turkey,3350000.0,39.9,5.95,5.95,258690.0,16315485.0,0.1221,8717910.0,...,0.065,2.6000,0.438,1992121.0,2021-12-31,1.467300e+06,0.057777,0.148402,0.010186,0.642450


We now have 255 banks:

In [4]:
df.shape

(255, 21)

Convert currencies to NZD and re-order the dataframe columns.

In [5]:
vals = ['MV', 'DWTA', 'EPS', 'P', 'BVPS', 'Tier1']
ids = ['NAME', 'country', 'NOSH']
dfc = df[ids].join(df[vals].div(0.6016).div(1000)).join(df[[x for x in df if x not in vals and x not in ids]])
df.sample(5)

,NAME,country,NOSH,PE,MTBV,PTBV,NI,RWA,Tier1Ratio,MV,...,EPS,P,BVPS,Tier1,date,BV,leverage,roe,roa,density
Type,,,,,,,,,,,,,,,,,,,,,
W:NDAS,NORDEA BANK,Finland,3965560.0,12.3,1.30,1.29,3757082.0,151575470.0,0.1910,42559780.0,...,0.88,10.7300,8.308,28948873.0,2021-12-31,3.294587e+07,0.057912,0.105922,0.006604,0.266439
N:VVL,VOSS VEKSEL- OG LANDMANDSBANK,Norway,2221.0,11.7,0.77,0.77,4448.0,291737.0,0.2013,44730.0,...,1.72,20.1400,26.306,58730.0,2021-12-31,5.842563e+04,0.101946,0.065384,0.007761,0.509048
IS:FIBIH,FIBI BANK HOLDING,Israel,35453.0,8.2,1.04,1.04,190709.0,25141988.0,0.1146,1424350.0,...,4.92,40.1761,38.587,2881547.0,2021-12-31,1.368025e+06,0.026918,0.127504,0.003753,0.494716
O:TYRO,BANK FUR TIROL UND VBG.,Austria,34031.0,13.2,0.59,0.58,80493.0,8213700.0,0.1362,1075390.0,...,2.39,31.6000,54.060,1119100.0,2021-12-31,1.839716e+06,0.129151,0.044210,0.005651,0.576613
CL:CRT,BANCO DE CREDITO E INVERSION,Chile,155887.0,8.0,0.86,0.86,537099.0,44679710.0,0.0999,4004590.0,...,2.98,23.7200,27.509,4463756.0,2021-12-31,4.288295e+06,0.060297,0.108328,0.007552,0.628231


Now select comparable banks that are similar to Kiwibank in size, leverage, performace, etc

In [6]:
dfc = dfc[dfc['DWTA'].between(10000,  150000)]  # Total assets
dfc = dfc[dfc['leverage'].between(0.02, 0.20)]
dfc = dfc[dfc['roe'].between(0.02,  0.25)]  # Exclude extreme performers
dfc = dfc[dfc['Tier1Ratio'].between(0.06,  0.25)]  # Include banks that meet capital requirements and have no extreme Tier 1 ratios
dfc = dfc[dfc['MTBV'].between(0.25,  5)]  # Exclude extreme Market to Book ratios
dfc = dfc[dfc['PE'].between(2, 20)] # Exclude extreme P/E ratios

dfc.sample(5)

,NAME,country,NOSH,MV,DWTA,EPS,P,BVPS,Tier1,PE,...,PTBV,NI,RWA,Tier1Ratio,date,BV,leverage,roe,roa,density
Type,,,,,,,,,,,,,,,,,,,,,
N:SVEG,SPAREBANKEN VEST,Norway,107323.0,1782.513298,38842.159242,0.001562,0.016606,0.011380,2863.783245,10.7,...,1.46,100238.0,9249771.0,0.1860,2021-12-31,734733.258,0.031443,0.137306,0.004290,0.395840
D:PCZ,PROCREDIT HOLDING,Germany,58898.0,783.228059,13653.848072,0.001961,0.013298,0.024167,1316.472739,6.8,...,0.55,79642.0,5600891.0,0.1410,2021-12-31,856318.022,0.104249,0.081161,0.009696,0.681858
TK:HLB,TURKIYE HALK BANKASI,Turkey,2473776.0,1228.008644,102528.690160,0.000055,0.000499,0.001946,6425.427194,9.0,...,0.26,121574.0,33398331.0,0.1157,2021-12-31,2896791.696,0.046964,0.028181,0.001971,0.541466
CZ:KOM,KOMERCNI BANKA,Czechia,190049.0,11886.253324,83229.780585,0.003773,0.062550,0.043745,6760.794548,16.6,...,1.43,512154.0,19491882.0,0.2087,2021-12-31,5001519.533,0.099888,0.086256,0.010229,0.389285
TK:AKB,AKBANK,Turkey,5200000.0,4120.994016,83930.442154,0.000199,0.000798,0.001607,8461.218418,4.1,...,0.49,803036.0,31467431.0,0.1618,2021-12-31,5028400.000,0.099587,0.124095,0.015904,0.623209


We now have 71 banks

In [7]:
dfc.shape

(71, 21)

Now use the data from the group of comparables to determine the value of Kiwibank.

Note that the value of the group of comparable firms increased by about 4 percent from the start of the year up to 22 August.

In [8]:
def valuation(df, nosh, driver, var, mv_appreciation):
    print(f'Value increase since start of the year: {mv_appreciation*100.0:4.3f} percent.')
    if var in ['MTBV', 'PTBV', 'PE']:
        k = stats.hmean(df[var], axis=0)  # According to Liu et al 2002, use the Harmonic mean, not the average
        if var == 'PE':
            print(f'Earnings Multiple: {k:4.2f}')
            print(f'EPS: ${driver / nosh:4.3f}.')
        elif var in ['MTBV', 'PTBV']:
            print(f'Book Multiple: {k:4.3f}')
            print(f'Book value per share: ${driver / nosh:4.3f}.')
        else:
            return
        print(f'Price per share: ${driver/nosh*k:4.5f}')
        valuation = driver * k / 1e6 * (1 + mv_appreciation)
        print(f'Valuation: ${valuation:4,.2f} in millions.')
    else:
        valuation = 'error'
    print('\n')
    return valuation


Use these inputs for Kiwibank

In [9]:
# From RBNZ Dashboard
profit_jun_22 = 131_300_000
book_value_jun_22 = 2_203_500_000

# Number of kiwibank shares
kiwi_nosh = 1_458_157_403
overall_delta_mv= 0.04_833_342

Determine the value

In [10]:
v1 = valuation(dfc, kiwi_nosh, profit_jun_22,     'PE',   overall_delta_mv)
v2 = valuation(dfc, kiwi_nosh, book_value_jun_22, 'MTBV', overall_delta_mv)  # using Market to Book ratios
v3 = valuation(dfc, kiwi_nosh, book_value_jun_22, 'PTBV', overall_delta_mv)  # using Price to Book ratios

print(f'Average Valuation: ${(v1+v2+v3)/3:4,.2f} in millions.')

Value increase since start of the year: 4.833 percent.
Earnings Multiple: 9.24
EPS: $0.090.
Price per share: $0.83224
Valuation: $1,272.19 in millions.


Value increase since start of the year: 4.833 percent.
Book Multiple: 0.783
Book value per share: $1.511.
Price per share: $1.18382
Valuation: $1,809.63 in millions.


Value increase since start of the year: 4.833 percent.
Book Multiple: 0.778
Book value per share: $1.511.
Price per share: $1.17575
Valuation: $1,797.30 in millions.


Average Valuation: $1,626.37 in millions.
